# MAST30034 Applied Data Science Project 2

## Geospatial Analysis

In [1]:
from pyspark.sql import SparkSession, functions as F, DataFrame
from pyspark.sql.functions import col
import geopandas as gpd
import folium
import pandas as pd

In [2]:
spark = (
    SparkSession.builder.appName("Geospatial_Analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/11 21:01:59 WARN Utils: Your hostname, Yis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.32.216 instead (on interface en0)
22/09/11 21:01:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/11 21:02:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/11 21:02:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/11 21:02:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/11 21:02:01 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
data = spark.read.parquet('../data/curated/final_data.parquet')

In [4]:
sf = gpd.read_file("../data/external/shapefile/SA2_2021_AUST_GDA2020.shp")

In [5]:
data = data.withColumn(
    'state',
    F.when(((col('postcode') >= 1000) & (col('postcode') <= 1999)) | ((col('postcode') >= 2000) & (col('postcode') <= 2599)) | ((col('postcode') >= 2619) & (col('postcode') < 2899)) | ((col('postcode') >= 2921) & (col('postcode') < 2999)), 'NSW')\
    .when(((col('postcode') >= 200) & (col('postcode') <= 299)) | ((col('postcode') >= 2600) & (col('postcode') <= 2618)) | ((col('postcode') >= 2900) & (col('postcode') < 2920)), 'ACT')\
    .when(((col('postcode') >= 3000) & (col('postcode') <= 3999)) | ((col('postcode') >= 8000) & (col('postcode') <= 8999)), 'VIC')\
    .when(((col('postcode') >= 4000) & (col('postcode') <= 4999)) | ((col('postcode') >= 9000) & (col('postcode') <= 9999)), 'QLD')\
    .when(((col('postcode') >= 5000) & (col('postcode') <= 5999)), 'SA')\
    .when(((col('postcode') >= 6000) & (col('postcode') <= 6999)), 'WA')\
    .when(((col('postcode') >= 7000) & (col('postcode') <= 7999)), 'TAS')\
    .otherwise('NT')
)
data = data.withColumn(
    'SA2_NAME_2021',
    F.when( (col('state') == 'VIC') & (col('SA2_NAME_2021').isNull()), 'Victoria')\
    .when( (col('state') == 'NSW') & (col('SA2_NAME_2021').isNull()), 'New South Wales')\
    .when( (col('state') == 'QLD') & (col('SA2_NAME_2021').isNull()), 'Queensland')\
    .when( (col('state') == 'NT') & (col('SA2_NAME_2021').isNull()), 'Northern Territory')\
    .when( (col('state') == 'WA') & (col('SA2_NAME_2021').isNull()), 'Western Austraia')\
    .when( (col('state') == 'SA') & (col('SA2_NAME_2021').isNull()), 'South Australia')\
    .when( (col('state') == 'TAS') & (col('SA2_NAME_2021').isNull()), 'Tasmania')\
    .otherwise('Australian Capital Territory')
)
data.limit(5)

22/09/11 21:02:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,take rate,revenue level,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
null,6915,3067,7470,47086412084,8.195033092624405,3b7b90dd-68a6-4f6...,2021-08-21,Eros Limited,[digital goods: b...,2.52,c,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,WA
null,6915,3067,7470,52065114842,106.91043190212513,49d4d2de-6783-435...,2021-08-14,Gravida Non PC,[digital goods: b...,3.98,b,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,WA
null,6915,3067,7470,13118172970,65.98639083026582,267b6bc3-3977-49d...,2021-08-15,Placerat Cras Inc...,"[gift, card, no...",4.57,b,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,WA
null,6915,3067,7470,35223308778,96.84997524124665,2f92ee68-38a7-4f1...,2021-07-15,Euismod In Corp.,"[books, periodic...",4.19,b,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,WA
null,6915,3067,7470,58392414752,30.000483014621604,f82dac78-e0ec-4da...,2021-08-27,Mattis Ornare Lec...,[florists supplie...,1.47,c,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,WA


In [6]:
data = data.withColumn("Median_tot_prsnl_inc_weekly", data.Median_tot_prsnl_inc_weekly.cast('float'))
data = data.withColumn("Median_rent_weekly", data.Median_rent_weekly.cast('float'))
data = data.withColumn("Median_mortgage_repay_monthly",data.Median_mortgage_repay_monthly.cast('float'))
data = data.withColumn("Median_age_persons",data.Median_age_persons.cast('float'))
data = data.withColumn("Median_tot_hhd_inc_weekly",data.Median_tot_hhd_inc_weekly.cast('float'))
data = data.withColumn("Average_household_size",data.Average_household_size.cast('float'))
data = data.withColumnRenamed("Completed Year 12", "Completed_Year_12")
data = data.withColumn("Completed_Year_12",data.Completed_Year_12.cast('float'))
data = data.withColumnRenamed("Did Not Attend School", "Did_Not_Attend_School")
data = data.withColumn("Did_Not_Attend_School",data.Did_Not_Attend_School.cast('float'))
data = data.withColumn("TOT_P_P",data.TOT_P_P.cast('float'))
data = data.withColumn("Age_0_4_yr_P",data.Age_0_4_yr_P.cast('float'))
data = data.withColumn("Age_5_14_yr_P",data.Age_5_14_yr_P.cast('float'))
data = data.withColumn("Age_15_19_yr_P",data.Age_15_19_yr_P.cast('float'))
data = data.withColumn("Age_20_24_yr_P",data.Age_20_24_yr_P.cast('float'))
data = data.withColumn("Age_25_34_yr_P",data.Age_25_34_yr_P.cast('float'))
data = data.withColumn("Age_35_44_yr_P",data.Age_35_44_yr_P.cast('float'))
data = data.withColumn("Age_45_54_yr_P",data.Age_45_54_yr_P.cast('float'))
data = data.withColumn("Age_55_64_yr_P",data.Age_55_64_yr_P.cast('float'))
data = data.withColumn("Age_65_74_yr_P",data.Age_65_74_yr_P.cast('float'))
data = data.withColumn("Age_75_84_yr_P",data.Age_75_84_yr_P.cast('float'))
data = data.withColumn("Age_75_84_yr_P",data.Age_75_84_yr_P.cast('float'))
data = data.withColumn("Age_75_84_yr_P",data.Age_75_84_yr_P.cast('float'))
data = data.withColumn("Age_85ov_P",data.Age_85ov_P.cast('float'))

In [7]:
SA2_data = data \
            .groupBy("SA2_CODE_2021") \
            .agg(
                F.mean("dollar_value").alias("mean_amount"),
                F.sum("dollar_value").alias("total_amount"),
                F.count("order_id").alias("order_volume"),
                F.mean("Median_tot_hhd_inc_weekly").alias("household_income"),
            ) \
            .orderBy("SA2_CODE_2021") \
            .toPandas()

In [8]:
SA2_data

,SA2_CODE_2021,mean_amount,total_amount,order_volume,household_income
0,None,166.118508,1.298538e+08,781694,NaN
1,101021007,192.732754,1.352907e+07,70196,1429.0
2,101021008,154.308454,4.709494e+05,3052,1989.0
3,101021009,154.308454,7.064241e+05,4578,1703.0
4,101021010,154.308454,1.177374e+06,7630,1796.0
...,...,...,...,...,...
2469,901031003,153.116404,2.448331e+06,15990,1825.0
2470,901041004,181.296365,1.103732e+06,6088,1184.0
2471,997979799,NaN,NaN,0,0.0
2472,999999499,NaN,NaN,0,0.0


In [9]:
# Code from MAST30034 Applied Data Science Tutorial 2
# Convert the geometry shape to to latitude and longitude
sf = sf[['SA2_CODE21','geometry']]
sf['geometry'] = sf['geometry'] \
                    .to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf = gpd.GeoDataFrame(
    pd.merge(SA2_data, sf, left_on='SA2_CODE_2021',right_on='SA2_CODE21', how='inner')
)
geoJSON = gdf[['SA2_CODE_2021', 'geometry']] \
    .drop_duplicates('SA2_CODE_2021').to_json()

In [10]:
m1 = folium.Map(location=[-33.8, 151.2],
                  tiles="cartodbpositron", zoom_start=4)

c1 = folium.Choropleth(
    geo_data=geoJSON,
    name='Income By SA2',
    data=gdf,
    columns=['SA2_CODE_2021', 'household_income'],
    key_on='properties.SA2_CODE_2021',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='Income By SA2'
)

c1.add_to(m1)

m1.save("../plots/income.html")

In [11]:
m2 = folium.Map(location=[-33.8, 151.2],
                  tiles="cartodbpositron", zoom_start=4)

c2 = folium.Choropleth(
    geo_data=geoJSON,
    name='Transaction Volume By SA2',
    data=gdf,
    columns=['SA2_CODE_2021', 'order_volume'],
    key_on='properties.SA2_CODE_2021',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='Transaction Volume By SA2'
)

c2.add_to(m2)

m2.save("../plots/volume.html")

In [12]:
m3 = folium.Map(location=[-33.8, 151.2],
                  tiles="cartodbpositron", zoom_start=4)

c3 = folium.Choropleth(
    geo_data=geoJSON,
    name='Income By SA2',
    data=gdf,
    columns=['SA2_CODE_2021', 'mean_amount'],
    key_on='properties.SA2_CODE_2021',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='Average Transaction Amount By SA2'
)

c3.add_to(m3)

m3.save("../plots/avg_amount.html")